# investigateEdgeCases

This notebook contains a suite of examples designed to test the DIBBs Record Linkage Algorithm against two critical categories of edge cases provided by LAC. It will use specific SQL queries and filtering to determine whether the algorithm can successfully handle the following scenarios:

* Persons with same address and similar names: parent/child, siblings, 
congregate living facilities (skilled nursing facility, jail/prison, etc.)
* Non-valid addresses that would not uniquely identify persons (e.g. 123 
Default St., Homeless, etc.)

If an edge case cannot correctly be handled, the discussion accompanying this notebook will outline why the algorithm failed for the case, identify possible data or algorithm solutions, and describe a proposed path forward.

## Setup
First, we'll configure the notebook to be able to access the UAT environment so that we can test on real production data.

In [ ]:
pip install psycopg2-binary azure-identity

In [ ]:
from azure.identity import DefaultAzureCredential
import psycopg2
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.types import StructType,StructField, StringType

In [ ]:
storage_account_name = "$STORAGE_ACCOUNT"
ECR_DELTA_TABLE_FILE_PATH = f"abfss://delta-tables@{storage_account_name}.dfs.core.windows.net/ecr-datastore" 

# Set your Key Vault information and Key Vault linked service
vault_name = "$KEY_VAULT"
vault_linked_service = "$KEY_VAULT_LINKED_SERVICE"
credential = DefaultAzureCredential()

# Database connection parameters
DB_NAME = "DibbsMpiDB"
DB_USER = "postgres"
DB_HOST = "$MPI_DB_HOST"
DB_PORT = "5432"
DB_TABLE = "patient"

# Get the secret value (password) from the previous step
db_password =  TokenLibrary.getSecret(vault_name,"mpi-db-password",vault_linked_service)

schema = StructType([
  StructField("patient_id", StringType(), True),
  StructField("person_id", StringType(), True),
  StructField("address", StringType(), True),
  StructField("birthdate", StringType(), True),
  StructField("city", StringType(), True),
  StructField("first_name", StringType(), True),
  StructField("last_name", StringType(), True),
  StructField("mrn", StringType(), True),
  StructField("sex", StringType(), True),
  StructField("state", StringType(), True),
  StructField("zip", StringType(), True),
])

In [ ]:
ADDR_1 = "NAME_AND_DOB_SCENARIO_HERE"
ADDR_2 = "MISSING_INFO_NON_MATCH_SCENARIO_HERE"
ADDR_3 = "SAME_NAME_CHILD_SCENARIO_HERE"
NAME_5 = "HOUSING_INSTABILITY_SCENARIO_HERE"

## Case 1: Same Address, Similar Name
These scenarios explore edge cases in which multiple patients live at the same address and have varying degrees of overlap in their name fields.

### Name and DOB, Correct Match + Non-Match
This case shows two patients who live together at the same address. All information is provided for both patients, except for MRN. Both patients have similar birthdays (they fuzzy match to each other), identical last names, and different first names (a husband and wife, most likely). The results show that the duplicate entries for the wife are correctly matched, while the patient record for the husband is correctly not matched.

In [ ]:
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=db_password,
    host=DB_HOST,
    port=DB_PORT
)

cur = conn.cursor()
cur.execute(f"""
    SELECT
    
    patient_id,
    person_id, 
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").line') as varchar), '"-[]','') as address,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.birthDate') as varchar), '"-[]','') as birthdate,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").city') as varchar), '"-[]','') as city,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.name[*] ?(@.use=="official").given') as varchar), '"-[]','') as first_name,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.name ?(@.use=="official").family') as varchar), '"-[]','') as last_name,
    TRANSLATE(CAST(jsonb_path_query_array(patient_resource, '$.identifier ?(@.type.coding[0].code=="MR").value') as varchar), '"-[]','') as mrn,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.gender') as varchar),'"-[]','') as sex,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").state') as varchar), '"-[]','') as state,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").postalCode') as varchar), '"-[]','') as zip

    FROM {DB_TABLE}

    WHERE patient_resource->'address' @? '$[*] ?(@.line=="${ADDR_1}")';

""")
data = cur.fetchall()
cur.close()
conn.close()

patient = spark.createDataFrame(data = data, schema = schema)
patient = patient.where(patient['last_name'].isNotNull())
patient.show()

### Missing Information, Correct Non-Match
This query shows the results of linkage on multiple distinct individuals who all live at the same address but are missing information. In this case, none of the provided individuals have a first or a last name on record. Since all three patients have the same address--a family, or perhaps individuals at a collective care facility--there may be concern that these patients will all be linked together. However, their different birthdays and different MRNs ensure that these patients are correctly not matched.

In [ ]:
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=db_password,
    host=DB_HOST,
    port=DB_PORT
)

cur = conn.cursor()
cur.execute(f"""
    SELECT
    
    patient_id,
    person_id, 
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").line') as varchar), '"-[]','') as address,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.birthDate') as varchar), '"-[]','') as birthdate,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").city') as varchar), '"-[]','') as city,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.name[*] ?(@.use=="official").given') as varchar), '"-[]','') as first_name,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.name ?(@.use=="official").family') as varchar), '"-[]','') as last_name,
    TRANSLATE(CAST(jsonb_path_query_array(patient_resource, '$.identifier ?(@.type.coding[0].code=="MR").value') as varchar), '"-[]','') as mrn,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.gender') as varchar),'"-[]','') as sex,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").state') as varchar), '"-[]','') as state,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").postalCode') as varchar), '"-[]','') as zip

    FROM {DB_TABLE}

    WHERE TRANSLATE(CAST(jsonb_path_query_array(patient_resource, '$.address[*] ?(@.use=="home").line') as varchar), '"-[]','')='${ADDR_2}';
""")
data = cur.fetchall()
cur.close()
conn.close()

patient = spark.createDataFrame(data = data, schema = schema)
patient.show()

### Incorrect Match of Child With Same Name
This case shows a scenairo in which a family of patients all live together at the same address. The child (evidenced by a different birthday) has the same name as one of their parents. While the linkage algorithm is able to correctly differentiate the parents, it is unable to separate the child from their parent of the same name.

In [ ]:
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=db_password,
    host=DB_HOST,
    port=DB_PORT
)

cur = conn.cursor()
cur.execute(f"""
    SELECT
    
    patient_id,
    person_id, 
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").line') as varchar), '"-[]','') as address,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.birthDate') as varchar), '"-[]','') as birthdate,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").city') as varchar), '"-[]','') as city,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.name[*] ?(@.use=="official").given') as varchar), '"-[]','') as first_name,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.name ?(@.use=="official").family') as varchar), '"-[]','') as last_name,
    TRANSLATE(CAST(jsonb_path_query_array(patient_resource, '$.identifier ?(@.type.coding[0].code=="MR").value') as varchar), '"-[]','') as mrn,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.gender') as varchar),'"-[]','') as sex,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").state') as varchar), '"-[]','') as state,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").postalCode') as varchar), '"-[]','') as zip

    FROM {DB_TABLE}

    WHERE patient_resource->'address' @? '$[*] ?(@.line=="{ADDR_3}")';
""")
data = cur.fetchall()
cur.close()
conn.close()

patient = spark.createDataFrame(data = data, schema = schema)
patient.sort('birthdate').show()

### What Went Wrong
Here, the algorithm was unable to separate the child from their parent of the same name. An examination of the birthdate and MRN field shows why. In all patients, the MRN field is missing entirely. We know that MRN is one of the best discriminative separators of patients, so in a close call like this, missing that information makes it difficult for the algorithm to correctly block the parent and child separately. Further, the birthdates of the child and their same-name parent are extremely similar. Both begin with a `19`, have a `0` in the 5th position, and contain a transposed `1` in the second to last column. The Jaro-Winkler similarity metric gives additional weight to strings that start the same and have common characters, meaning that these birthdates look mathematically similar enough that the algorithm considered them a match. If the birthdates were more different--such as the child being born after the year 2000, or in a different enough month-day combination--the algorithm would have judged the birthdays different enough to not match them. 

### Proposed Fix
Given the importance of this parent-child edge case, we propose a post-processing heuristic that will apply a more granular filter to birthdates for the case when multiple people of the same name live at the same address. Rather than fuzzy match these birthdays with the same threshold as a more general case, we will increase the scoring threshold required to match on birthday, so that even if the first names are similar, that won't be enough to classify two patients as the same.

## Non-Valid Address That Does Not Uniquely Identify
This scenario explores how the linkage algorithm handles patient records with recorded "non-valid" addresses (such as default addresses) that may not uniquely identify patients that they pertain to.

### Correct Separation of Patients at Default Address
This case demonstrates the algorithm's ability to filter past a default address to separate multiple patients who live there according to other present, more discriminating information.

In [ ]:
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=db_password,
    host=DB_HOST,
    port=DB_PORT
)

cur = conn.cursor()
cur.execute(f"""
    SELECT
    
    patient_id,
    person_id, 
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").line') as varchar), '"-[]','') as address,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.birthDate') as varchar), '"-[]','') as birthdate,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").city') as varchar), '"-[]','') as city,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.name[*] ?(@.use=="official").given') as varchar), '"-[]','') as first_name,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.name ?(@.use=="official").family') as varchar), '"-[]','') as last_name,
    TRANSLATE(CAST(jsonb_path_query_array(patient_resource, '$.identifier ?(@.type.coding[0].code=="MR").value') as varchar), '"-[]','') as mrn,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.gender') as varchar),'"-[]','') as sex,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").state') as varchar), '"-[]','') as state,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").postalCode') as varchar), '"-[]','') as zip

    FROM {DB_TABLE}

    WHERE patient_resource->'address' @? '$[*] ?(@.line=="123 DEFAULT ST")';
""")
data = cur.fetchall()
cur.close()
conn.close()

patient = spark.createDataFrame(data = data, schema = schema)
patient = patient.where(patient["first_name"].isNotNull()).where(patient["address"] == "123 DEFAULT ST")
patient.sort('last_name').show(10)

### Incorrect Non-Match of Same Patient Across Multiple Default Addresses
This case illustrates how the algorithm mistakenly classifies a single patient with multiple default address values as multiple individuals. Here, one or more of the address values is intendend to represent housing instability or homelessness.

In [ ]:
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=db_password,
    host=DB_HOST,
    port=DB_PORT
)

cur = conn.cursor()
cur.execute(f"""
    SELECT
    
    patient_id,
    person_id, 
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").line') as varchar), '"-[]','') as address,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.birthDate') as varchar), '"-[]','') as birthdate,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").city') as varchar), '"-[]','') as city,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.name[*] ?(@.use=="official").given') as varchar), '"-[]','') as first_name,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.name ?(@.use=="official").family') as varchar), '"-[]','') as last_name,
    TRANSLATE(CAST(jsonb_path_query_array(patient_resource, '$.identifier ?(@.type.coding[0].code=="MR").value') as varchar), '"-[]','') as mrn,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.gender') as varchar),'"-[]','') as sex,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").state') as varchar), '"-[]','') as state,
    TRANSLATE(CAST(jsonb_path_query(patient_resource, '$.address[*] ?(@.use=="home").postalCode') as varchar), '"-[]','') as zip

    FROM {DB_TABLE}

    WHERE TRANSLATE(CAST(jsonb_path_query_array(patient_resource, '$.name[*] ?(@.use=="official").given') as varchar), '"-[]','')='{NAME_5}';
""")
data = cur.fetchall()
cur.close()
conn.close()

patient = spark.createDataFrame(data = data, schema = schema)
patient = patient.where(patient['first_name'].isNotNull())
patient.show()

### What Went Wrong
As odd as it may sound, from the algorithm's point of view, nothing. The last record in this block may have the same name and and birthdate as the previous duplicate records, but its address line and even its city is completely different. Further, there is no provided MRN to connect these individuals across different cities. However, data collection and representation among vulnerable populations can often be incomplete, so we can't rely on MRN to differentiate patients with transient housing or other instability. Moreover, while the algorithm sees the values `PT STATES HOMELESS` and `123 DEFAULT ST` as both real and distinct, we know better. Both of these are merely "default" placeholder values representing housing instability. Therefore, they shouldn't really be treated as different for the purposes of linking.

### Proposed Fix
Much like the edge case involving a child with the same name as a parent, our proposed solution here is a stronger incorporation of post-processing checks. Instead of simply checking whether two incoming addresses are "different," we propose to implement some more intelligent "default" checking into the linkage algorithm. That way, when two patients are being compared and are found to have these default or placeholder values, we can apply more granular criteria on the information that is provided, such as name and date of birth. We would love to discuss the details of the post-processing LAC is currently doing so that we can intelligently incorporate it into our solution.